### Importing the flow data.

In [0]:
#Import required libraries.
import requests as rq
import pandas as pd
import os

In [0]:
#API URL to get water flow stations on River Parrett.
api_url = "http://environment.data.gov.uk/hydrology/id/stations.json"
params = {"observedProperty": "waterFlow","riverName": "River Parrett"}

# Request data from the API using 'get'.
response = requests.get(api_url, params=params)
response.raise_for_status()  # Stop if there's an error and defines the error.
stations = response.json().get("items", []) #Converts the response into a Python dictionary and gets the "items" key.

# Extract relevant data into a list using list comprehension, checking it is a dictionary and is complete. [<expression> for <item> in <iterable> if <condition>]
station_data = [{"Station Name": station.get("label"),"GUID": station.get("stationGuid"),"Latitude": station.get("lat"),"Longitude": station.get("long"),"River Served": station.get("riverName", "N/A"),"Catchment": station.get("catchmentName", "N/A")} for station in stations if isinstance(station, dict) and station.get("notation")]

# Convert to DataFrame.
df_stations = pd.DataFrame(station_data)


In [0]:
#Initialise a list.
readings = []

 #Loop over unique station GUIDs
for guid in df_stations["GUID"].dropna().unique():
    #API URL.
    url = "http://environment.data.gov.uk/hydrology/data/readings.json"
    #Parameters for the request. 
    params = {"station": guid,"observedProperty": "waterFlow","periodName": "15min","mineq-date": "2019-12-19","max-date": "2024-11-25","_limit": 2000000}
    #Parameters for the request. 
    response = rq.get(url, params=params)
    response.raise_for_status()  # Stop if there's an error and defines the error.
    items = response.json().get("items", []) #Returns a Python dictionary of the items.
    # Append readings to the list, adding the GUID for reference.
    for item in items:
        item["GUID"] = guid
        readings.append(item)

#Create a DataFrame from the collected readings.
df_readings = pd.DataFrame(readings)


In [0]:
df_stations

,Station Name,GUID,Latitude,Longitude,River Served,Catchment
0,Chiselborough,7998bf73-641d-4084-b00c-ca6989f2ba2b,50.926859,-2.768693,River Parrett,N/A


In [0]:
#Investigate how many unique GUIDs (and therefore stations) there are. 
print(df_readings['GUID'].nunique())

In [0]:
df_readings

,measure,date,dateTime,value,quality,GUID
0,{'@id': 'http://environment.data.gov.uk/hydrol...,2019-12-19,2019-12-19T00:00:00,14.228,Good,7998bf73-641d-4084-b00c-ca6989f2ba2b
1,{'@id': 'http://environment.data.gov.uk/hydrol...,2019-12-19,2019-12-19T00:15:00,15.746,Good,7998bf73-641d-4084-b00c-ca6989f2ba2b
2,{'@id': 'http://environment.data.gov.uk/hydrol...,2019-12-19,2019-12-19T00:30:00,18.130,Good,7998bf73-641d-4084-b00c-ca6989f2ba2b
3,{'@id': 'http://environment.data.gov.uk/hydrol...,2019-12-19,2019-12-19T00:45:00,21.454,Good,7998bf73-641d-4084-b00c-ca6989f2ba2b
4,{'@id': 'http://environment.data.gov.uk/hydrol...,2019-12-19,2019-12-19T01:00:00,24.321,Good,7998bf73-641d-4084-b00c-ca6989f2ba2b
...,...,...,...,...,...,...
173083,{'@id': 'http://environment.data.gov.uk/hydrol...,2024-03-13,2024-03-13T22:00:00,1.562,Good,7998bf73-641d-4084-b00c-ca6989f2ba2b
173084,{'@id': 'http://environment.data.gov.uk/hydrol...,2024-03-13,2024-03-13T22:15:00,1.562,Good,7998bf73-641d-4084-b00c-ca6989f2ba2b
173085,{'@id': 'http://environment.data.gov.uk/hydrol...,2024-03-13,2024-03-13T22:30:00,1.562,Good,7998bf73-641d-4084-b00c-ca6989f2ba2b
173086,{'@id': 'http://environment.data.gov.uk/hydrol...,2024-03-14,2024-03-14T04:00:00,1.499,Good,7998bf73-641d-4084-b00c-ca6989f2ba2b


In [0]:
#Rename GUID to station.
df_readings.rename(columns={"GUID": "station"}, inplace=True)

In [0]:
# Make sure the directory exists
os.makedirs("Data", exist_ok=True)

# Save the DataFrame to the defined path
df_readings.to_csv(os.path.join("Data", "Imported_API_Flow.csv"), index=False)

# Print the absolute path of the saved CSV
print(f"CSV saved to {os.path.abspath(os.path.join('Data', 'Imported_API_Flow.csv'))}")